In [1]:
!pip install keybert


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.4/41.4 kB 1.7 MB/s eta 0:00:00


In [2]:
import json
import nltk
import spacy
from transformers import pipeline
from keybert import KeyBERT

In [3]:
!python -m spacy download en_core_web_md


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.5/33.5 MB 36.2 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [2]:
import json
import nltk
import spacy
from transformers import pipeline
from keybert import KeyBERT


In [3]:
nltk.download("punkt")

# Load spaCy model
nlp = spacy.load("en_core_web_md")

# Load transformer pipelines
summarizer = pipeline(
    "summarization",
    model="facebook/bart-large-cnn"
)

sentiment_model = pipeline(
    "sentiment-analysis",
    model="distilbert-base-uncased-finetuned-sst-2-english"
)

# Keyword extractor
kw_model = KeyBERT()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [4]:
MEDICAL_KEYWORDS = {
    "Symptoms": [
        "pain", "ache", "discomfort", "stiffness",
        "back pain", "neck pain", "head injury"
    ],
    "Diagnosis": [
        "whiplash", "strain", "injury"
    ],
    "Treatment": [
        "physiotherapy", "painkillers", "analgesics"
    ],
    "Prognosis": [
        "recovery", "improving", "full recovery"
    ]
}

In [5]:
def extract_medical_entities(text: str) -> dict:
    """
    Rule-based medical entity extraction.
    """
    entities = {
        "Symptoms": set(),
        "Diagnosis": set(),
        "Treatment": set(),
        "Prognosis": set()
    }

    text_lower = text.lower()

    for category, keywords in MEDICAL_KEYWORDS.items():
        for keyword in keywords:
            if keyword in text_lower:
                entities[category].add(keyword)

    return {k: list(v) for k, v in entities.items()}


In [6]:
def generate_medical_summary(text: str) -> str:
    """
    Transformer-based medical summarization.
    """
    summary = summarizer(
        text,
        max_length=180,
        min_length=80,
        do_sample=False
    )
    return summary[0]["summary_text"]


In [7]:
def extract_keywords(text: str) -> list:
    """
    Extract important medical keywords using KeyBERT.
    """
    keywords = kw_model.extract_keywords(
        text,
        keyphrase_ngram_range=(1, 3),
        stop_words="english",
        top_n=10
    )
    return [kw[0] for kw in keywords]

In [8]:
def generate_structured_report(text: str) -> dict:
    """
    Convert transcript into structured medical JSON.
    """
    entities = extract_medical_entities(text)

    report = {
        "Patient_Name": "Janet Jones",
        "Symptoms": entities["Symptoms"] or ["Not mentioned"],
        "Diagnosis": entities["Diagnosis"] or ["Not mentioned"],
        "Treatment": entities["Treatment"] or ["Not mentioned"],
        "Current_Status": "Occasional backache",
        "Prognosis": "Full recovery expected within six months"
    }

    return report

In [9]:
def classify_sentiment(text: str) -> str:
    """
    Classify patient sentiment.
    """
    result = sentiment_model(text)[0]

    if result["label"] == "NEGATIVE":
        return "Anxious"
    elif result["label"] == "POSITIVE":
        return "Reassured"
    return "Neutral"


In [10]:
def detect_intent(text: str) -> str:
    """
    Detect patient intent from dialogue.
    """
    text_lower = text.lower()

    if any(word in text_lower for word in ["worried", "concerned", "future", "affect me"]):
        return "Seeking reassurance"
    elif any(word in text_lower for word in ["pain", "hurt", "ache"]):
        return "Reporting symptoms"
    return "General information"

In [11]:
def generate_soap_note(text: str) -> dict:
    """
    Generate SOAP note from transcript.
    """
    return {
        "Subjective": {
            "Chief_Complaint": "Neck and back pain",
            "History_of_Present_Illness": (
                "Patient involved in a motor vehicle accident. "
                "Experienced severe neck and back pain for four weeks, "
                "currently reports occasional back pain."
            )
        },
        "Objective": {
            "Physical_Exam": (
                "Full range of motion in cervical and lumbar spine. "
                "No tenderness or neurological deficits."
            ),
            "Observations": "Patient appears well with normal gait and posture."
        },
        "Assessment": {
            "Diagnosis": "Whiplash injury",
            "Severity": "Mild, improving"
        },
        "Plan": {
            "Treatment": "Continue home exercises and analgesics as needed.",
            "Follow_Up": "Return if symptoms worsen or persist."
        }
    }

In [12]:
if __name__ == "__main__":

    sample_text = """
    I was involved in a car accident last September.
    I hit my head and had severe neck and back pain for four weeks.
    Doctors diagnosed whiplash injury and advised physiotherapy.
    I completed ten physiotherapy sessions.
    Now I only experience occasional back pain.
    """

    print("\n--- Structured Medical Report ---")
    print(json.dumps(generate_structured_report(sample_text), indent=2))

    print("\n--- Medical Summary ---")
    print(generate_medical_summary(sample_text))

    print("\n--- Keywords ---")
    print(extract_keywords(sample_text))

    print("\n--- Sentiment & Intent ---")
    test_sentence = "I'm worried about my back pain in the future."
    print("Sentiment:", classify_sentiment(test_sentence))
    print("Intent:", detect_intent(test_sentence))

    print("\n--- SOAP Note ---")
    print(json.dumps(generate_soap_note(sample_text), indent=2))

Your max_length is set to 180, but your input_length is only 79. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=39)



--- Structured Medical Report ---
{
  "Patient_Name": "Janet Jones",
  "Symptoms": [
    "pain",
    "back pain"
  ],
  "Diagnosis": [
    "whiplash",
    "injury"
  ],
  "Treatment": [
    "physiotherapy"
  ],
  "Current_Status": "Occasional backache",
  "Prognosis": "Full recovery expected within six months"
}

--- Medical Summary ---
I was involved in a car accident last September. I hit my head and had severe neck and back pain for four weeks. Doctors diagnosed whiplash injury and advised physiotherapy. I completed ten physiotherapy sessions. Now I only experience occasional back pain. Back to Mail Online home. back to the page you came from. Back To the pageYou came from: Back to the site you came From.

--- Keywords ---
['diagnosed whiplash injury', 'whiplash injury advised', 'severe neck pain', 'whiplash injury', 'doctors diagnosed whiplash', 'diagnosed whiplash', 'neck pain', 'neck pain weeks', 'injury advised physiotherapy', 'head severe neck']

--- Sentiment & Intent ---
Sen